In [23]:
import math
import random

In [24]:
class Tree(object):
    def __init__(self, data, left = None, right = None):
        self.left = left
        self.right = right
        self.data = data
    
    def __str__(self):
        return str(self.data)
    
    def getTerminals(self, terminals = []):
        if self.left != None:
            terminals = self.left.getTerminals(terminals)
        if self.right != None:
            terminals = self.right.getTerminals(terminals)
        if self.left == None and self.right == None:
            terminals.append(self.data)
        return terminals
    
    def printAll(self, numSpaces = 0):
        if self.right != None:
            self.right.printAll(numSpaces + 10)
            print("\n")
        
        for i in range(numSpaces):
            print(end = " ")
        print(self)
        
        if self.left != None:
            print("\n")
            self.left.printAll(numSpaces + 10)

In [29]:
class Jet(Tree):
    E_INITIAL = 8000
    E_CRIT = 0.17
    
    def randZ():
        y = random.random()
        z = 2 ** y - 1
        return z
    
    def randTheta():
        y = random.random()
        theta = (1 + math.pi / 2) ** y - 1
        return theta
    
    def randPhi():
        return math.pi * random.random()
    
    def pInitial():
        e = - Jet.E_INITIAL * math.log(1 - random.random())
        theta = math.pi * random.random()
        phi = 2 * math.pi * random.random()
        p = [0, 0, 0, 0]
        
        p[0] = e
        p[1] = e * math.sin(theta) * math.cos(phi)
        p[2] = e * math.sin(theta) * math.sin(phi)
        p[3] = e * math.cos(theta)
        return p
    
    def radiate(p):
        rad = [0, 0, 0, 0]
        z = Jet.randZ()
        theta = Jet.randTheta()
        phi = Jet.randPhi()
        s = math.sqrt(p[1] ** 2 + p[2] ** 2)
        
        rad[0] = z * p[0]
        if s != 0:
            rad[1] = z * ((p[1] * p[3] * math.sin(theta) * math.cos(phi) - p[2] * p[0] * math.sin(theta) * math.sin(phi)) / s + p[1] * math.cos(theta))
            rad[2] = z * ((p[2] * p[3] * math.sin(theta) * math.cos(phi) + p[1] * p[0] * math.sin(theta) * math.sin(phi)) / s + p[2] * math.cos(theta))
        else:
            rad[1] = z * (p[3] * math.sin(theta) * math.cos(phi) + p[1] * math.cos(theta))
            rad[2] = z * (p[0] * math.sin(theta) * math.sin(phi) + p[2] * math.cos(theta)) 
        rad[3] = z * (p[3] * math.cos(theta) - s * math.sin(theta) * math.cos(phi))
        return rad
    
    def __init__(self, data):
        super().__init__(data)
        if data[0] > Jet.E_CRIT:
            rad = Jet.radiate(data)
            f = [0, 0, 0, 0]
            
            for i in range(4):
                f[i] = data[i] - rad[i]
            self.left = Jet(rad)
            self.right = Jet(f)

In [30]:
a = Jet(Jet.pInitial())

In [36]:
b = a.getTerminals()
sumTheta = 0
count = 0
for i in b:
    sumTheta += math.atan(math.sqrt(i[1] ** 2 + i[2] ** 2) / i[3])
    count += 1
print(count)
print(sumTheta * 180 / (count * math.pi))

262073
3.1230138700151286
